# Social Laws Use Case Demo

This notebook shows a demonstration of a realistic use case for social laws. In this demo we will synthesize a social law for a multi-agent pickup and delivery problem.


The maps will be specified as a list of strings where 
* \* denotes an obstacle
* a lower case letter (a-z) denotes the initial position of an agent
* an upper case letter (A-Z) denotes the goal position of the corresponding agent



In [40]:
small_map = [
"****",
"*BA*",
"*ab*",
"****"
]



We now define a function which converts the given map into a multi agent planning problem

In [41]:
from unified_planning.shortcuts import *
from social_laws.ma_problem_waitfor import MultiAgentProblemWithWaitfor
from unified_planning.model.multi_agent import *
import string
up.environment.get_environment().credits_stream = None


def mapToMaProblem(map):
    problem = MultiAgentProblemWithWaitfor("MAPD")

    loc = UserType("loc")
    
    # Environment     
    connected = Fluent('connected', BoolType(), l1=loc, l2=loc)        
    problem.ma_environment.add_fluent(connected, default_initial_value=False)

    free = Fluent('free', BoolType(), l=loc)
    problem.ma_environment.add_fluent(free, default_initial_value=True)




    # Agents
    at = Fluent('at', BoolType(), l1=loc)

    move = InstantaneousAction('move', l1=loc, l2=loc)
    l1 = move.parameter('l1')
    l2 = move.parameter('l2')
    move.add_precondition(at(l1))
    move.add_precondition(free(l2))
    move.add_precondition(connected(l1,l2))
    move.add_effect(at(l2),True)
    move.add_effect(free(l2), False)
    move.add_effect(at(l1), False)
    move.add_effect(free(l1), True)    


    locmap = {}

    for i,line in enumerate(map):
        for j,cell in enumerate(line):
            locname = "l_" + str(i) + "_" + str(j)
            locobj = Object(locname, loc)
            problem.add_object(locobj)
            locmap[i,j] = locobj
            if cell == "*":
                problem.set_initial_value(free(locobj), False)
            elif cell in string.ascii_lowercase:
                agentname = "a_" + cell
                agent = Agent(agentname, problem)
                problem.add_agent(agent)
                agent.add_fluent(at, default_initial_value=False)
                agent.add_action(move)
                problem.set_initial_value(Dot(agent, at(locobj)), True)
    
    for i,line in enumerate(map):
        for j,cell in enumerate(line):
            if cell in string.ascii_uppercase:
                agentname = "a_" + cell.lower()
                locobj = locmap[i,j]
                agent = problem.agent(agentname)
                problem.add_goal(Dot(agent, at(locobj)))


    for i,j in locmap.keys():
        for dx,dy in [(-1,0),(+1,0),(0,-1),(0,+1)]:
            if (i+dx, j+dy) in locmap.keys():        
                problem.set_initial_value(connected(locmap[i,j], locmap[i+dx,j+dy]), True)        

    return problem


problem = mapToMaProblem(small_map)
print(problem)




problem name = MAPD

types = [loc]

environment fluents = [
  bool connected[l1=loc, l2=loc]
  bool free[l=loc]
]

agents = [
  Agent name = a_a

fluents = [
 bool at[l1=loc]
]

actions = [
 action move(loc l1, loc l2) {
    preconditions = [
      at(l1)
      free(l2)
      connected(l1, l2)
    ]
    effects = [
      at(l2) := true
      free(l2) := false
      at(l1) := false
      free(l1) := true
    ]
    simulated effect = None
  }
]

goals = [
]


  Agent name = a_b

fluents = [
 bool at[l1=loc]
]

actions = [
 action move(loc l1, loc l2) {
    preconditions = [
      at(l1)
      free(l2)
      connected(l1, l2)
    ]
    effects = [
      at(l2) := true
      free(l2) := false
      at(l1) := false
      free(l1) := true
    ]
    simulated effect = None
  }
]

goals = [
]


]

objects = [
  loc: [l_0_0, l_0_1, l_0_2, l_0_3, l_0_4, l_0_5, l_0_6, l_0_7, l_0_8, l_0_9, l_0_10, l_0_11, l_0_12, l_0_13, l_1_0, l_1_1, l_1_2, l_1_3, l_1_4, l_1_5, l_1_6, l_1_7, l_1_8, l_1_9, l_1_10,

In [ ]:
## We can now synthesize a social law for the map

In [ ]:
from social_laws.synthesis import get_gbfs_social_law_generator

generator = get_gbfs_social_law_generator()
robust_sl = generator.generate_social_law(problem)

print(robust_sl)

In [ ]:
## Let's see what the plan under the social law is

In [ ]:
from unified_planning.social_law.robustness_checker import SocialLawRobustnessChecker
slrc = SocialLawRobustnessChecker()

new_problem = robust_sl.compile(problem).problem
presult = slrc.solve(new_problem)
print(presult.status)
print(presult.plan)

In [ ]:
## Now let's try a slightly bigger map

In [ ]:
map = [
"*****",
"*A b*",
"*a B*",
"*****"
]

problem = mapToMaProblem(map)
generator = get_gbfs_social_law_generator()
robust_sl = generator.generate_social_law(problem)

print(robust_sl)

In [ ]:
new_problem = robust_sl.compile(problem).problem
presult = slrc.solve(new_problem)
print(presult.status)
print(presult.plan)

In [ ]:
## And now an even larger map

In [ ]:
large_map = [
"**************",
"* B        A *",
"*  *******   *",
"* a        b *",
"**************"
]

problem = mapToMaProblem(large_map)
generator = get_gbfs_social_law_generator()
robust_sl = generator.generate_social_law(problem)

print(robust_sl)

In [ ]:
new_problem = robust_sl.compile(problem).problem
presult = slrc.solve(new_problem)
print(presult.status)
print(presult.plan)

In [ ]:
## Analysis

The larger map is not solved in a reasonable amount of time.

As the original paper on social laws in planning [1] suggests, incorporating the FF planners as well an a strong unsolvability planner should help improve this.

[1] Automated Verification of Social Law Robustness in STRIPS
Erez Karpas, Alexander Shleyfman, and Moshe Tennenholtz
ICAPS 2017